# CH07-02: 嵌入層 (Embeddings)

**課程目標:**
- 理解詞嵌入 (Word Embeddings) 的核心概念
- 掌握 Token Embeddings 與 Position Embeddings 的實作
- 比較 Word2Vec, GloVe 與 Transformer Embeddings 的差異
- 實戰訓練與評估詞向量品質

**學習時間:** 約 60 分鐘

**前置知識:**
- 詞向量基礎概念 (CH05-02)
- Transformer 架構概覽 (CH07-01)

---

## 📚 目錄

1. [為什麼需要詞嵌入?](#1)
2. [Token Embeddings 實作](#2)
3. [Positional Embeddings 深入](#3)
4. [Word2Vec vs Transformer Embeddings](#4)
5. [實戰: 訓練與評估詞向量](#5)
6. [進階技巧與優化](#6)

---

In [ ]:
# 環境設定與套件導入
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 設定隨機種子
np.random.seed(42)

print("✅ 環境設定完成")
print(f"NumPy 版本: {np.__version__}")

<a id="1"></a>
## 1. 為什麼需要詞嵌入?

### 1.1 從 One-Hot 到詞嵌入的演進

#### 問題: 如何表示詞?

**方法 1: One-Hot Encoding (獨熱編碼)**

```python
詞彙表: ["貓", "狗", "鳥"]

"貓" = [1, 0, 0]
"狗" = [0, 1, 0]
"鳥" = [0, 0, 1]
```

**三大問題:**
1. ❌ **維度災難**: 詞彙 10 萬 → 10 萬維向量!
2. ❌ **稀疏性**: 99.999% 都是 0
3. ❌ **無語義**: "貓" 和 "狗" 同樣遙遠 (余弦相似度 = 0)

---

**方法 2: 詞嵌入 (Word Embeddings)**

```python
"貓" = [0.8, -0.3, 0.5, ..., 0.2]  # 256 維稠密向量
"狗" = [0.7, -0.2, 0.6, ..., 0.3]  # 相似!
"鳥" = [0.3,  0.8, 0.1, ..., -0.5] # 不同
```

**三大優勢:**
1. ✅ **低維稠密**: 通常 256-1024 維
2. ✅ **捕捉語義**: 相似詞向量接近
3. ✅ **可運算**: 支援向量運算 ("國王" - "男人" + "女人" ≈ "女王")

---

### 1.2 Transformer 中的雙重嵌入

Transformer 使用**兩種嵌入**的組合:

$$
\text{Input} = \text{Token Embedding} + \text{Positional Embedding}
$$

1. **Token Embedding**: 表示詞的**語義**
2. **Positional Embedding**: 表示詞的**位置**

**為什麼需要位置嵌入?**

- Self-Attention 是**無序的** (permutation-invariant)
- 必須顯式告知模型詞的順序

---

In [ ]:
# 示範: One-Hot vs 詞嵌入
vocab = ["貓", "狗", "鳥", "魚", "兔子"]
vocab_size = len(vocab)
embedding_dim = 3  # 簡化為 3 維以便視覺化

# One-Hot 編碼
one_hot = np.eye(vocab_size)

# 模擬詞嵌入 (實際應透過訓練獲得)
# 設計: 貓、狗接近; 鳥、魚接近; 兔子介於中間
embeddings = np.array([
    [0.8, 0.2, 0.1],   # 貓
    [0.7, 0.3, 0.15],  # 狗 (與貓相似)
    [0.1, 0.8, 0.7],   # 鳥
    [0.15, 0.75, 0.8], # 魚 (與鳥相似)
    [0.5, 0.5, 0.4],   # 兔子 (中間)
])

# 視覺化
fig = plt.figure(figsize=(16, 6))

# One-Hot 編碼
ax1 = fig.add_subplot(121)
sns.heatmap(one_hot, annot=True, fmt='.0f', cmap='Greys', 
            xticklabels=[f'維度{i+1}' for i in range(vocab_size)],
            yticklabels=vocab, cbar_kws={'label': 'Value'})
ax1.set_title('One-Hot 編碼 (稀疏, 無語義)', fontsize=14, fontweight='bold')

# 詞嵌入
ax2 = fig.add_subplot(122, projection='3d')
colors = ['red', 'red', 'blue', 'blue', 'green']
for i, word in enumerate(vocab):
    ax2.scatter(embeddings[i, 0], embeddings[i, 1], embeddings[i, 2], 
                c=colors[i], s=200, alpha=0.6)
    ax2.text(embeddings[i, 0], embeddings[i, 1], embeddings[i, 2], 
             word, fontsize=12, fontweight='bold')
ax2.set_xlabel('Dimension 1', fontsize=11)
ax2.set_ylabel('Dimension 2', fontsize=11)
ax2.set_zlabel('Dimension 3', fontsize=11)
ax2.set_title('詞嵌入 (稠密, 有語義)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔍 觀察:")
print("  - One-Hot: 每個詞都是正交向量 (無相似度)")
print("  - 詞嵌入: 相似詞 (貓&狗, 鳥&魚) 在空間中接近")

<a id="2"></a>
## 2. Token Embeddings 實作

### 2.1 嵌入層 (Embedding Layer) 原理

**嵌入層本質:** 一個**查找表** (Lookup Table)

```python
Embedding Matrix: (vocab_size, embedding_dim)

              維度 1   維度 2   ...  維度 d
詞 0 ("the")   0.23    -0.45   ...   0.12
詞 1 ("cat")   0.78     0.12   ...  -0.34
詞 2 ("sat")  -0.23     0.89   ...   0.56
...
詞 N          ...
```

**查找過程:**

```python
輸入 token ID: 1 ("cat")
    ↓
查找 Embedding Matrix 第 1 行
    ↓
輸出: [0.78, 0.12, ..., -0.34]
```

---

### 2.2 PyTorch 風格實作

In [ ]:
# 從零實作 Embedding Layer
class SimpleEmbedding:
    """
    簡化版嵌入層實作
    
    Args:
        vocab_size: 詞彙表大小
        embedding_dim: 嵌入向量維度
    """
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        # 初始化嵌入矩陣 (隨機初始化, 訓練時會更新)
        self.weight = np.random.randn(vocab_size, embedding_dim) * 0.1
        
    def forward(self, token_ids):
        """
        前向傳播: 查表獲取嵌入向量
        
        Args:
            token_ids: shape (batch_size, seq_len) 或 (seq_len,)
            
        Returns:
            embeddings: shape (batch_size, seq_len, embedding_dim) 或 (seq_len, embedding_dim)
        """
        return self.weight[token_ids]
    
    def __call__(self, token_ids):
        return self.forward(token_ids)


# 測試
vocab_size = 10000  # 詞彙表大小
embedding_dim = 256  # BERT-base 使用 768, GPT-2 使用 768, GPT-3 使用 12288

embedding_layer = SimpleEmbedding(vocab_size, embedding_dim)

# 模擬輸入: 一個句子的 token IDs
# 例如: "The cat sat on the mat" → [20, 45, 123, 56, 20, 89]
token_ids = np.array([20, 45, 123, 56, 20, 89])

# 獲取嵌入
embeddings = embedding_layer(token_ids)

print(f"輸入 Token IDs 形狀: {token_ids.shape}")
print(f"輸出 Embeddings 形狀: {embeddings.shape}")
print(f"\n嵌入矩陣參數量: {vocab_size * embedding_dim:,} ({vocab_size:,} × {embedding_dim})")
print(f"\n第一個 token (ID=20) 的嵌入向量 (前 10 維):")
print(embeddings[0, :10])

# 驗證: 相同 token 的嵌入應該相同
print(f"\n✅ 驗證: token[0] 和 token[4] 嵌入是否相同? {np.allclose(embeddings[0], embeddings[4])}")

### 2.3 嵌入層的訓練

**關鍵問題:** 如何學習好的嵌入?

#### 兩種訓練方式:

1. **預訓練 (Pre-trained)**
   - 在大規模語料上預先訓練 (Word2Vec, GloVe)
   - 遷移到下游任務
   - 優點: 利用海量數據, 泛化能力強

2. **端到端訓練 (End-to-End)**
   - 作為模型一部分,隨任務一起訓練
   - Transformer 通常採用此方式
   - 優點: 針對特定任務優化

---

### 2.4 嵌入維度的選擇

| 模型 | 詞彙量 | 嵌入維度 | 參數量 |
|------|--------|----------|--------|
| Word2Vec | 100K | 300 | 30M |
| BERT-base | 30K | 768 | 23M |
| BERT-large | 30K | 1024 | 31M |
| GPT-2 | 50K | 768 | 38M |
| GPT-3 | 50K | 12288 | 614M |

**權衡:**
- 維度 ↑ → 表達能力 ↑, 計算成本 ↑
- 維度 ↓ → 訓練速度 ↑, 可能欠擬合

**經驗法則:** 256-1024 維適合大多數任務

---

In [ ]:
# 實驗: 嵌入維度對參數量的影響
vocab_sizes = [10000, 30000, 50000, 100000]
embedding_dims = [128, 256, 512, 768, 1024]

# 計算參數量
param_counts = np.array([[v * d for d in embedding_dims] for v in vocab_sizes])

# 視覺化
fig, ax = plt.subplots(figsize=(12, 6))

for i, vocab_size in enumerate(vocab_sizes):
    ax.plot(embedding_dims, param_counts[i] / 1e6, 'o-', 
            label=f'Vocab: {vocab_size:,}', linewidth=2, markersize=8)

ax.set_xlabel('Embedding Dimension', fontsize=12)
ax.set_ylabel('Parameters (Millions)', fontsize=12)
ax.set_title('嵌入層參數量分析', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 洞察:")
print("  - GPT-3 (50K vocab, 12288 dim) = 614M 參數僅在嵌入層!")
print("  - 嵌入層參數量 = 詞彙量 × 嵌入維度")
print("  - 大型模型中,嵌入層可占總參數量的 5-20%")

<a id="3"></a>
## 3. Positional Embeddings 深入

### 3.1 為什麼 Attention 需要位置訊息?

**問題示範:**

```
句子 A: "狗咬人"
句子 B: "人咬狗"  ← 順序不同,語義完全相反!
```

但 Self-Attention 的計算是:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$$

這個公式對詞的順序**不敏感**!

```python
Attention(["狗", "咬", "人"]) == Attention(["人", "咬", "狗"])  # 完全相同!
```

**解決方案:** 加入位置編碼

---

### 3.2 兩種位置編碼方式

#### 方法 1: 正弦位置編碼 (Sinusoidal) - Transformer 原論文

$$
\text{PE}_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d}}\right)
$$

$$
\text{PE}_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d}}\right)
$$

**優點:**
- ✅ 不需訓練 (固定函數)
- ✅ 可處理任意長度序列
- ✅ 相對位置可計算

---

#### 方法 2: 可學習位置編碼 (Learned) - BERT, GPT

```python
position_embeddings = Embedding(max_seq_len, embedding_dim)
```

**優點:**
- ✅ 可針對任務優化
- ✅ 實作簡單

**缺點:**
- ❌ 需要設定最大長度 (max_seq_len)
- ❌ 無法直接處理超長序列

---

In [ ]:
# 完整實作兩種位置編碼

class SinusoidalPositionalEncoding:
    """正弦位置編碼 (Transformer 原論文)"""
    
    def __init__(self, d_model, max_len=5000):
        self.d_model = d_model
        
        # 預計算位置編碼
        pe = np.zeros((max_len, d_model))
        position = np.arange(0, max_len).reshape(-1, 1)
        
        # 計算除數項
        div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
        
        # Sin for even indices
        pe[:, 0::2] = np.sin(position * div_term)
        # Cos for odd indices
        pe[:, 1::2] = np.cos(position * div_term)
        
        self.pe = pe
    
    def __call__(self, seq_len):
        """返回序列長度為 seq_len 的位置編碼"""
        return self.pe[:seq_len]


class LearnedPositionalEncoding:
    """可學習位置編碼 (BERT/GPT)"""
    
    def __init__(self, max_len, d_model):
        self.max_len = max_len
        self.d_model = d_model
        
        # 隨機初始化 (訓練時會更新)
        self.pe = np.random.randn(max_len, d_model) * 0.02
    
    def __call__(self, seq_len):
        """返回序列長度為 seq_len 的位置編碼"""
        if seq_len > self.max_len:
            raise ValueError(f"序列長度 {seq_len} 超過最大長度 {self.max_len}")
        return self.pe[:seq_len]


# 測試與比較
d_model = 512
max_len = 100

sin_pe = SinusoidalPositionalEncoding(d_model, max_len)
learned_pe = LearnedPositionalEncoding(max_len, d_model)

# 獲取位置編碼
sin_encoding = sin_pe(50)
learned_encoding = learned_pe(50)

# 視覺化比較
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 正弦編碼
im1 = axes[0].imshow(sin_encoding.T, cmap='RdBu', aspect='auto', vmin=-1, vmax=1)
axes[0].set_xlabel('Position', fontsize=12)
axes[0].set_ylabel('Dimension', fontsize=12)
axes[0].set_title('正弦位置編碼 (Sinusoidal)\n固定、可外推', fontsize=13, fontweight='bold')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

# 可學習編碼
im2 = axes[1].imshow(learned_encoding.T, cmap='RdBu', aspect='auto', vmin=-1, vmax=1)
axes[1].set_xlabel('Position', fontsize=12)
axes[1].set_ylabel('Dimension', fontsize=12)
axes[1].set_title('可學習位置編碼 (Learned)\n可訓練、固定長度', fontsize=13, fontweight='bold')
plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

print("\n📊 特性對比:")
print("\n正弦編碼:")
print("  - 規律的波形模式 (低維高頻, 高維低頻)")
print("  - 可處理任意長度")
print("  - 使用: Transformer 原論文, T5")
print("\n可學習編碼:")
print("  - 隨機初始化,需訓練")
print("  - 受限於 max_len")
print("  - 使用: BERT, GPT, GPT-2, GPT-3")

### 3.3 位置編碼的數學直覺

#### 為什麼正弦函數有效?

**關鍵性質:** 相對位置可以線性表示

$$
\text{PE}(pos + k) = f(\text{PE}(pos), k)
$$

利用三角恆等式:

$$
\sin(\alpha + \beta) = \sin\alpha\cos\beta + \cos\alpha\sin\beta
$$

模型可以學習「向前/向後 k 個位置」的關係!

---

In [ ]:
# 視覺化: 位置編碼的相對位置特性
d_model = 128
pe = SinusoidalPositionalEncoding(d_model, max_len=200)
encodings = pe(200)

# 計算不同位置對之間的余弦相似度
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# 選擇幾個參考位置
reference_positions = [0, 50, 100, 150]
similarity_matrix = np.zeros((len(reference_positions), 200))

for i, ref_pos in enumerate(reference_positions):
    for target_pos in range(200):
        similarity_matrix[i, target_pos] = cosine_similarity(
            encodings[ref_pos], encodings[target_pos]
        )

# 視覺化
plt.figure(figsize=(14, 6))

for i, ref_pos in enumerate(reference_positions):
    plt.plot(similarity_matrix[i], label=f'參考位置 {ref_pos}', linewidth=2)

plt.xlabel('Target Position', fontsize=12)
plt.ylabel('Cosine Similarity', fontsize=12)
plt.title('位置編碼的余弦相似度分析\n(相似度隨距離遞減)', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🔍 觀察:")
print("  - 相鄰位置相似度高 (峰值在自己位置)")
print("  - 距離越遠,相似度越低")
print("  - 呈現週期性模式 (正弦函數特性)")

<a id="4"></a>
## 4. Word2Vec vs Transformer Embeddings

### 4.1 詞嵌入演進史

```
2013 → Word2Vec (Mikolov et al.)
       - CBOW, Skip-gram
       - 靜態嵌入 (每個詞只有一個向量)

2014 → GloVe (Pennington et al.)
       - 全局詞-詞共現矩陣
       - 靜態嵌入

2017 → ELMo (Peters et al.)
       - 雙向 LSTM
       - 上下文相關嵌入 (開始!)

2018 → BERT (Devlin et al.)
       - Transformer Encoder
       - 強大的上下文嵌入

2019+ → GPT-2/3, RoBERTa, T5, ...
        - Transformer 統治 NLP
```

---

### 4.2 靜態 vs 上下文嵌入

#### 問題: 一詞多義

```python
句子 1: "我去銀行存錢"      ← "銀行" = 金融機構
句子 2: "我在河銀行釣魚"    ← "銀行" = 河岸
```

**Word2Vec/GloVe (靜態):**
```python
"銀行" → [0.23, -0.45, 0.67, ...]  # 永遠相同!
```

**BERT/GPT (上下文):**
```python
"銀行" in "我去銀行存錢"   → [0.78, 0.12, -0.23, ...]
"銀行" in "我在河銀行釣魚" → [-0.34, 0.89, 0.45, ...]  # 不同!
```

---

### 4.3 詳細對比

| 特性 | Word2Vec/GloVe | Transformer (BERT/GPT) |
|------|----------------|------------------------|
| **嵌入類型** | 靜態 | 上下文相關 |
| **訓練目標** | 預測鄰近詞 | Masked LM / CLM |
| **參數量** | ~100M | 100M - 175B |
| **訓練數據** | Wikipedia | Internet-scale |
| **一詞多義** | ❌ 無法處理 | ✅ 完美處理 |
| **訓練成本** | 低 (CPU 可訓練) | 高 (需大量 GPU) |
| **適用場景** | 簡單分類、聚類 | 幾乎所有 NLP 任務 |

---

In [ ]:
# 模擬: 靜態 vs 上下文嵌入

# 靜態嵌入 (Word2Vec 風格)
word2vec_bank = np.array([0.5, 0.3, -0.2, 0.8])  # "銀行" 永遠相同

# 上下文嵌入 (BERT 風格 - 簡化模擬)
def contextual_embedding(word, context):
    """
    模擬上下文相關嵌入
    (實際 BERT 會通過 Transformer 計算)
    """
    base = np.array([0.5, 0.3, -0.2, 0.8])
    
    if "存錢" in context or "金融" in context:
        # 金融語境
        context_shift = np.array([0.3, -0.1, 0.2, 0.0])
    elif "河" in context or "釣魚" in context:
        # 自然語境
        context_shift = np.array([-0.3, 0.4, 0.1, -0.3])
    else:
        context_shift = np.zeros(4)
    
    return base + context_shift

# 兩個不同語境
context1 = "我去銀行存錢"
context2 = "我在河銀行釣魚"

bert_bank_1 = contextual_embedding("銀行", context1)
bert_bank_2 = contextual_embedding("銀行", context2)

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 靜態嵌入
axes[0].bar(['維度1', '維度2', '維度3', '維度4'], word2vec_bank, color='steelblue', alpha=0.7)
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[0].set_ylim(-1, 1)
axes[0].set_title('Word2Vec: 靜態嵌入\n"銀行" 永遠相同', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Value', fontsize=11)

# 上下文嵌入
x = np.arange(4)
width = 0.35
axes[1].bar(x - width/2, bert_bank_1, width, label='"我去銀行存錢"', alpha=0.7)
axes[1].bar(x + width/2, bert_bank_2, width, label='"我在河銀行釣魚"', alpha=0.7)
axes[1].set_xticks(x)
axes[1].set_xticklabels(['維度1', '維度2', '維度3', '維度4'])
axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[1].set_ylim(-1, 1)
axes[1].set_title('BERT: 上下文嵌入\n"銀行" 隨語境改變', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Value', fontsize=11)
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()

# 計算相似度
cos_sim = cosine_similarity(bert_bank_1, bert_bank_2)
print(f"\n📐 兩種語境下 '銀行' 的余弦相似度: {cos_sim:.3f}")
print("   (< 1.0 表示不完全相同,成功捕捉語境差異!)")

<a id="5"></a>
## 5. 實戰: 訓練與評估詞向量

### 5.1 使用 Gensim 訓練 Word2Vec

In [ ]:
# 訓練簡單的 Word2Vec 模型
try:
    from gensim.models import Word2Vec
    GENSIM_AVAILABLE = True
except ImportError:
    GENSIM_AVAILABLE = False
    print("⚠️  Gensim 未安裝,跳過此部分")
    print("   安裝: pip install gensim")

if GENSIM_AVAILABLE:
    # 準備訓練語料 (簡化示例)
    sentences = [
        ['國王', '統治', '國家'],
        ['女王', '統治', '王國'],
        ['男人', '是', '人類'],
        ['女人', '是', '人類'],
        ['國王', '是', '男人'],
        ['女王', '是', '女人'],
        ['貓', '抓', '老鼠'],
        ['狗', '追', '貓'],
        ['鳥', '在', '天空', '飛'],
        ['魚', '在', '水', '裡', '游'],
    ] * 100  # 重複以增加訓練數據
    
    # 訓練 Word2Vec
    print("🚀 訓練 Word2Vec 模型...")
    model = Word2Vec(
        sentences=sentences,
        vector_size=50,     # 嵌入維度
        window=3,           # 上下文窗口
        min_count=1,        # 最小詞頻
        workers=4,          # 並行線程
        sg=1,               # Skip-gram (1) or CBOW (0)
        epochs=50
    )
    
    print("✅ 訓練完成!")
    print(f"\n詞彙量: {len(model.wv)}")
    print(f"嵌入維度: {model.wv.vector_size}")
    
    # 測試詞向量
    print("\n🔍 詞向量相似度測試:")
    test_pairs = [
        ('國王', '女王'),
        ('男人', '女人'),
        ('貓', '狗'),
        ('鳥', '魚'),
        ('國王', '貓'),  # 不相關
    ]
    
    for word1, word2 in test_pairs:
        try:
            sim = model.wv.similarity(word1, word2)
            print(f"  {word1:4s} ↔ {word2:4s}: {sim:.3f}")
        except KeyError:
            print(f"  {word1} 或 {word2} 不在詞彙表中")
    
    # 經典類比任務: 國王 - 男人 + 女人 ≈ 女王
    print("\n👑 經典類比: 國王 - 男人 + 女人 = ?")
    try:
        result = model.wv.most_similar(
            positive=['女人', '國王'], 
            negative=['男人'], 
            topn=3
        )
        for word, score in result:
            print(f"  {word}: {score:.3f}")
    except:
        print("  語料太小,無法完成類比")

### 5.2 詞向量品質評估

#### 三種評估方式:

1. **內在評估 (Intrinsic)**
   - 詞相似度任務
   - 詞類比任務
   - 快速,但與下游任務相關性低

2. **外在評估 (Extrinsic)**
   - 在實際任務上測試 (分類、NER 等)
   - 慢,但最準確

3. **視覺化評估**
   - t-SNE / PCA 降維可視化
   - 直觀,但主觀

---

In [ ]:
# 視覺化詞向量 (使用 t-SNE 降維)
if GENSIM_AVAILABLE:
    # 獲取所有詞向量
    words = list(model.wv.index_to_key)
    vectors = np.array([model.wv[word] for word in words])
    
    # t-SNE 降維到 2D
    print("🎨 使用 t-SNE 降維到 2D...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(words)-1))
    vectors_2d = tsne.fit_transform(vectors)
    
    # 視覺化
    plt.figure(figsize=(12, 10))
    
    # 繪製所有詞
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], alpha=0.5, s=100)
    
    # 標註詞
    for i, word in enumerate(words):
        plt.annotate(word, 
                    xy=(vectors_2d[i, 0], vectors_2d[i, 1]),
                    xytext=(5, 2),
                    textcoords='offset points',
                    fontsize=11,
                    fontweight='bold')
    
    plt.xlabel('t-SNE Dimension 1', fontsize=12)
    plt.ylabel('t-SNE Dimension 2', fontsize=12)
    plt.title('Word2Vec 詞向量 t-SNE 視覺化\n(相似詞應該聚集在一起)', 
              fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\n💡 觀察:")
    print("  - 相似詞 (如 '國王' 和 '女王') 應該靠近")
    print("  - 不同語義類別 (如動物 vs 人物) 應該分離")

<a id="6"></a>
## 6. 進階技巧與優化

### 6.1 嵌入層的初始化策略

#### 常見初始化方法:

1. **隨機初始化**
   ```python
   embeddings = np.random.randn(vocab_size, d_model) * 0.01
   ```
   - 簡單,但需要更多訓練時間

2. **Xavier/Glorot 初始化**
   ```python
   limit = np.sqrt(6 / (vocab_size + d_model))
   embeddings = np.random.uniform(-limit, limit, (vocab_size, d_model))
   ```
   - 保持梯度穩定

3. **預訓練嵌入**
   ```python
   embeddings = load_pretrained_embeddings("glove.6B.300d.txt")
   ```
   - 遷移學習,冷啟動快

---

### 6.2 嵌入層的正則化

#### Dropout on Embeddings

```python
embeddings = embedding_layer(token_ids)
embeddings = dropout(embeddings, p=0.1)  # 隨機丟棄 10%
```

**作用:** 防止過擬合,提升泛化能力

---

### 6.3 子詞嵌入 (Subword Embeddings)

**問題:** 罕見詞、新詞、拼寫錯誤?

**解決方案:** 使用子詞單元 (Subword Units)

#### 三種主流方法:

1. **Byte Pair Encoding (BPE)** - GPT系列
   ```
   "tokenization" → ["token", "ization"]
   ```

2. **WordPiece** - BERT
   ```
   "tokenization" → ["token", "##ization"]
   ```

3. **SentencePiece** - T5, LLaMA
   ```
   "tokenization" → ["▁token", "ization"]
   ```

**優點:**
- ✅ 處理未登錄詞 (OOV)
- ✅ 減小詞彙表大小
- ✅ 多語言友好

---

In [ ]:
# 示範: 簡單的 BPE 分詞
try:
    from transformers import GPT2Tokenizer
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    TRANSFORMERS_AVAILABLE = False
    print("⚠️  Transformers 未安裝,跳過此示範")
    print("   安裝: pip install transformers")

if TRANSFORMERS_AVAILABLE:
    # 載入 GPT-2 Tokenizer (使用 BPE)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    # 測試句子
    sentences = [
        "tokenization",
        "unbelievable",
        "Transformer",
        "COVID-19",  # 新詞
    ]
    
    print("\n🔤 BPE 子詞分詞示範 (GPT-2):")
    print("=" * 50)
    
    for sent in sentences:
        tokens = tokenizer.tokenize(sent)
        token_ids = tokenizer.encode(sent)
        print(f"\n原文: {sent}")
        print(f"  ├─ Tokens: {tokens}")
        print(f"  └─ IDs:    {token_ids}")
    
    print("\n💡 觀察:")
    print("  - 常見詞保持完整 (如 'Transformer')")
    print("  - 罕見詞分解為子詞 (如 'token' + 'ization')")
    print("  - 新詞也能處理 (如 'COVID-19')")

---

## 📚 本課總結

### 核心要點回顧:

1. **詞嵌入基礎:**
   - One-Hot → 詞嵌入的演進
   - 低維稠密向量表示語義
   - 嵌入層 = 可訓練的查找表

2. **Transformer 雙重嵌入:**
   - Token Embeddings: 表示詞的語義
   - Positional Embeddings: 表示詞的位置
   - 輸入 = Token Emb + Position Emb

3. **位置編碼兩種方式:**
   - 正弦編碼: 固定、可外推 (Transformer 原論文)
   - 可學習編碼: 可訓練、固定長度 (BERT, GPT)

4. **靜態 vs 上下文嵌入:**
   - Word2Vec/GloVe: 靜態,無法處理一詞多義
   - BERT/GPT: 上下文相關,捕捉語境

5. **進階技巧:**
   - 初始化策略 (隨機、Xavier、預訓練)
   - 正則化 (Dropout)
   - 子詞嵌入 (BPE, WordPiece, SentencePiece)

---

## 🎯 下節預告

**CH07-03: 注意力機制 (Attention Mechanism)**

我們將深入探討:
- Self-Attention 數學推導
- Multi-Head Attention 實作
- Scaled Dot-Product Attention
- Attention 可視化與解釋性
- 實戰: 從零實作完整 Attention 層

---

## 📖 延伸閱讀

1. **論文:**
   - [Efficient Estimation of Word Representations](https://arxiv.org/abs/1301.3781) (Word2Vec)
   - [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf)
   - [Deep Contextualized Word Representations](https://arxiv.org/abs/1802.05365) (ELMo)

2. **教學資源:**
   - [Word Embeddings Explained](https://www.tensorflow.org/text/guide/word_embeddings)
   - [The Illustrated Word2Vec](http://jalammar.github.io/illustrated-word2vec/)

3. **實作教學:**
   - [Gensim Word2Vec Tutorial](https://radimrehurek.com/gensim/models/word2vec.html)
   - [Hugging Face Tokenizers](https://huggingface.co/docs/tokenizers/index)

---

### 🙋 課後練習

1. 使用自己的語料訓練 Word2Vec 模型
2. 比較不同初始化策略對訓練的影響
3. 實作一個簡單的 BPE 分詞器

---

**課程資訊:**
- **作者:** iSpan NLP Team
- **版本:** v1.0
- **最後更新:** 2025-10-17
- **授權:** MIT License (僅供教學使用)